# Entregable 1 (RNN)
Por Carlos Sánchez Polo
Asignatura Deep Learning

## Enunciado:

Aplicación de SimpleRNN a:
1. Dataset IMDB de Keras (https://keras.io/api/datasets/imdb/),
2. Otro Dataset de Keras o TensorFlow (https://keras.io/api/datasets/, https://www.tensorflow.org/datasets/catalog/overviewhl=%20es#all_datasets)